### Load Pre-Processed Data

In [8]:
# retrieve stored values

%store -r x_train 
%store -r x_test 
%store -r y_train 
%store -r y_test 
%store -r yy 
%store -r le


### Setting up CNN

In [9]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics
import tensorflow as tf

if len(tf.config.list_physical_devices('GPU')):
    print("GPU detected")
else:
    print("CPU detected, this may take a while")

GPU detected


In [10]:
num_rows = 40
num_columns = 174
num_channels = 1

x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax')) 

### Compiling Model

In [11]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

In [12]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 39, 173, 16)       80        
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 19, 86, 16)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 19, 86, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 18, 85, 32)        2080      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 9, 42, 32)         0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 9, 42, 32)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 41, 64)        

In [13]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 
import tqdm.notebook as tqdm
from tqdm.keras import TqdmCallback

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='models/weights.best.basic_mlp.hdf5', 
                               verbose=0, save_best_only=True)
start = datetime.now()
    
model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer,TqdmCallback(verbose=2)], verbose = 0)

duration = datetime.now() - start
print("Training completed in time: ", duration)



Training completed in time:  0:02:04.623296


### Results from training


In [14]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9972798824310303
Testing Accuracy:  0.926159143447876
